In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('classic')
import numpy as np
import datetime as dt
from datetime import timedelta
import simulate
from debt import Debt
import math
import subprocess

pd.options.display.float_format = '{:20,.5f}'.format
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 50)

In [2]:
spx = pd.read_csv('^GSPC.csv', index_col=0)

start = dt.date(2020, 1, 1)
end = dt.date(2080, 1, 31)

Market = simulate.Market(spx.iloc[-7500:, -2], start, end)

In [3]:
Market.garch(log=False)
#plt.plot(Market.garch(log=False)['Price'])

Iteration:      1,   Func. Count:      9,   Neg. LLF: 219034.93771858286
Iteration:      2,   Func. Count:     21,   Neg. LLF: 262807.76526202995
Iteration:      3,   Func. Count:     31,   Neg. LLF: 181565.29570811347
Iteration:      4,   Func. Count:     40,   Neg. LLF: 2606331.78638033
Iteration:      5,   Func. Count:     51,   Neg. LLF: 9680.033867601682
Iteration:      6,   Func. Count:     60,   Neg. LLF: 446230.4465759386
Iteration:      7,   Func. Count:     71,   Neg. LLF: 9579.92531052516
Iteration:      8,   Func. Count:     80,   Neg. LLF: 9669.387750022546
Iteration:      9,   Func. Count:     89,   Neg. LLF: 9570.089646588482
Iteration:     10,   Func. Count:     98,   Neg. LLF: 12905.495278355076
Iteration:     11,   Func. Count:    107,   Neg. LLF: 9564.312001905633
Iteration:     12,   Func. Count:    115,   Neg. LLF: 9564.315592701609
Iteration:     13,   Func. Count:    124,   Neg. LLF: 9564.307763835614
Iteration:     14,   Func. Count:    133,   Neg. LLF: 9564.306

,data,volatility,errors,Price
2020-01-01,0.02370,1.80268,2.33907,100.00000
2020-01-02,-0.01052,1.71143,-1.08275,98.94832
2020-01-03,-0.00487,1.69002,-0.51779,98.46673
2020-01-06,-0.01592,1.62027,-1.62311,96.89910
2020-01-07,0.01835,1.68867,1.80435,98.67761
...,...,...,...,...
2080-01-25,-0.00617,0.69277,-0.64759,"5,062.06604"
2080-01-26,-0.00028,0.72284,-0.05947,"5,060.62855"
2080-01-29,-0.00343,0.69607,-0.37457,"5,043.24565"
2080-01-30,0.00382,0.68957,0.35085,"5,062.50699"


## Checking stability of garch model

In [ ]:
empirical_mu = ((spx.iloc[-7500:, -2][-1]/spx.iloc[-7500:, -2][0])**(1/7500)-1)*100
market = Market.garch(log=False, mu_override = empirical_mu)

## Setting up functions to calculate performance

In [ ]:
def determine_investment(phase, pv_u, tv_u, s, td, pi_rf, dst, g, period):
    
    # returns cash, new_equity, new_debt
    
    if phase == 1:
        # Check if gearing cap has been reached
        equity = tv_u+s-td
        if td > (equity*g):
            new_debt = 0
        else:
            #desired_debt = min(g*(equity+s), dst-equity)
            #new_debt = max(0, desired_debt-td)
            new_debt = nd(g, s, tv_u, td, dst, period)
        return 0, s, new_debt
    
    if phase == 2: 
        stocks_sold = max(pv_u-dst, 0)
        debt_repayment = min(td, s + stocks_sold)
        repayment_left = debt_repayment
        
        if 'Nordnet' in debt_available.keys():
            repayment = min(debt_repayment, debt_available['Nordnet'].debt_amount)
            debt_available['Nordnet'].prepayment(repayment)
            repayment_left = debt_repayment - repayment
            
        if 'SU' in debt_available.keys():
            debt_available['SU'].prepayment(repayment_left)
            
        leftover_savings = max(s-debt_repayment - stocks_sold, 0)
        return 0, leftover_savings, -debt_repayment
    
    if phase == 3:
        return 0, s, 0
    
    if phase == 4:
        desired_cash = (1-pi_rf)*(tv_u+s)
        desired_savings = (pi_rf)*(tv_u+s)
        change_in_stock = desired_savings - pv_u
        return desired_cash, change_in_stock, 0

In [ ]:
# Function assumes monthly periods

def nd(g, s, tv_u, td, dst, period):
    
    equity = tv_u+s-td
    total_desired_debt = min(g/(g+1)*dst, equity*g)
    remaning_debt_needed = max(0, total_desired_debt - td)
    
    SU_amount, Nordnet_amount = 0, 0
    
    
    if period <= 60 and 'SU' in debt_available.keys():
        # Has SU already been taken?
        SU_amount = min(3234, remaning_debt_needed)
        debt_available['SU'].add_debt(SU_amount)

        remaning_debt_needed - SU_amount

    if 'Nordnet' in debt_available.keys():
        # Has Nordnet already been taken?
        Nordnet_amount = min(max(0, g*equity), remaning_debt_needed)
        debt_available['Nordnet'].add_debt(Nordnet_amount)


    return SU_amount + Nordnet_amount

In [ ]:
def interest():
    interest_bill = 0
    for debt in debt_available.keys():
        try:
            interest_bill += debt_available[debt].calculate_interest()
        except:
            pass
        
    return interest_bill

In [ ]:
def phase_check(phase, pi_rf, pi_rm, pi_hat, td):
    if phase == 4:
        return 4
    
    if td > 0:
        #has target not been reached?
        if pi_hat < pi_rm and phase <= 1:
            return 1
        else:
            # if target has been reached once and debt remains, stay in phase 2
            return 2
    
    #if target has been reached and no debt remains
    #is the value still above the target?
    if pi_hat < pi_rf:
        return 3
    else:
        return 4

In [ ]:
def calc_pi(gamma, sigma, mr, rate, cost = 0):
    return (mr - cost - rate)/(gamma * sigma)

In [ ]:
#def (gearing_cap, su, ):

## Combining with investment profile

In [ ]:
def calculate_return(savings, market_returns, gearing_cap, pi_rf, pi_rm, rf):
    
    wipeout = False
    
    # Running controls
    assert len(savings) == len(market), 'Investment plan should be same no of periods as market' 
    
    # Setting up constants and dataframe for calculation
    ses = savings.sum()     # Possibly add more sophisticated discounting
    ist = pi_rm*ses
    columns = ['period', 'savings', 'cash', 'new_equity', 'new_debt', 'total_debt', 'nip', 'pv_p', 
               'interest', 'market_returns', 'pv_u', 'tv_u', 'equity', 'dst', 'phase', 'pi_hat', 'ses', 'g_hat', 'SU_debt', 'Nordnet_debt']
    
    pp = pd.DataFrame(np.zeros((len(savings), len(columns))), columns = columns)
        
    pp['ses'] = ses
    pp['period'] = range(len(savings))
    pp['market_returns'] = market_returns
    pp['savings'] = savings
    pp.loc[0, 'market_returns'] = 0
    
    
    # Initializing debt objects
    if 'SU' in debt_available.keys():
        debt_available['SU'] = Debt(rate_structure = [[0, 0, 0.04]], rate_structure_type='relative', initial_debt = 0)
    if 'Nordnet' in debt_available.keys(): 
        debt_available['Nordnet'] = Debt(rate_structure = [[0, .4, 0.02], [.4, .6, 0.03], [.6, 0, 0.07]], rate_structure_type='relative', initial_debt = 0)

                      
    # Period 0 primo
    pp.loc[0, 'cash'] = 0
    pp.loc[0, 'new_equity'] = pp.loc[0, 'savings']
    pp.loc[0, 'new_debt'] = pp.loc[0, 'new_equity']*gearing_cap
    pp.loc[0, 'total_debt'] = pp.loc[0, 'new_debt']
    pp.loc[0, 'SU_debt'] = min(pp.loc[0, 'new_debt'], 3248)
    pp.loc[0, 'Nordnet_debt'] = max(0, pp.loc[0, 'new_debt'] - 3248)
    pp.loc[0, 'nip'] = pp.loc[0, 'new_debt'] + pp.loc[0, 'new_equity']
    pp.loc[0, 'pv_p'] = pp.loc[0, 'nip']
    pp.loc[0, 'pi_hat'] =  pp.loc[0, 'pv_p']/ses
    
    # Period 0 ultimo
    pp.loc[0, 'interest'] = pp.loc[0, 'new_debt']*max(interest(), 0)                   
    pp.loc[0, 'pv_u'] = pp.loc[0, 'pv_p']                 
    pp.loc[0, 'tv_u'] = pp.loc[0, 'pv_u'] + pp.loc[0, 'cash']
    pp.loc[0, 'equity'] = pp.loc[0, 'tv_u'] - pp.loc[0, 'total_debt']
    pp.loc[0, 'dst'] = ist
    pp.loc[0, 'phase'] = 1
    
              
    
    # Looping over all reminaning periods
    for i in range(1, len(savings)):
 
        # Period t > 0 primo
        if not(pp.loc[i-1, 'tv_u'] <= 0 and (pp.loc[i-1, 'interest'] > pp.loc[i, 'savings'])):

            pp.loc[i, 'cash'] = pp.loc[i-1, 'cash']*(1+rf) 
            pp.loc[i, 'cash'], pp.loc[i, 'new_equity'], pp.loc[i, 'new_debt'] = determine_investment(
                                                                                         pp.loc[i-1, 'phase'], pp.loc[i-1, 'pv_u'], 
                                                                                         pp.loc[i-1, 'tv_u'], pp.loc[i, 'savings'], pp.loc[i-1, 'total_debt'],
                                                                                         pi_rf, pp.loc[i-1, 'dst'], gearing_cap, pp.loc[i, 'period'])

            if 'SU' in debt_available.keys():
                pp.loc[i, 'SU_debt'] = debt_available['SU'].debt_amount
            if 'Nordnet' in debt_available.keys():
                pp.loc[i, 'Nordnet_debt'] = debt_available['Nordnet'].debt_amount
                
            pp.loc[i, 'total_debt'] = pp.loc[i-1, 'total_debt'] + pp.loc[i, 'new_debt']
            pp.loc[i, 'nip'] = pp.loc[i, 'new_equity'] + max(0, pp.loc[i, 'new_debt'])
            pp.loc[i, 'pv_p'] = pp.loc[i-1, 'pv_u'] + pp.loc[i, 'nip']        


            # Period t > 0 ultimo   
            if pp.loc[i, 'period'] == 60 and 'SU' in debt_available.keys():
                    debt_available['SU'].rate_structure = [[0, 0, 0.01]]
    
            pp.loc[i, 'interest'] = max(interest(), 0)
            pp.loc[i, 'pv_u'] = pp.loc[i, 'pv_p']*(1+pp.loc[i, 'market_returns'])-pp.loc[i, 'interest']
            pp.loc[i, 'tv_u'] = pp.loc[i, 'pv_u'] + pp.loc[i, 'cash']
            pp.loc[i, 'equity'] =  pp.loc[i, 'tv_u'] - pp.loc[i, 'total_debt']
            pp.loc[i, 'pi_hat'] = min(pp.loc[i, 'pv_u']/ses, pp.loc[i, 'pv_u']/pp.loc[i, 'tv_u'])

            pp.loc[i, 'phase'] = phase_check(pp.loc[i-1, 'phase'], pi_rf, pi_rm, pp.loc[i, 'pi_hat'], pp.loc[i, 'total_debt'])
            target_pi = pi_rm if pp.loc[i-1, 'phase'] < 3 else pi_rf
            pp.loc[i, 'dst'] = max(pp.loc[i, 'tv_u']*target_pi, ist)  # Moving stock target
            #pp.loc[i, 'dst'] = max(pp.loc[i-1, 'dst'], max(pp.loc[i, 'tv_u']*target_pi, ist))  # Locked stock target at highest previous position       
       
        else:
            wipeout = True
            print('Warning: Fatal wipeout')

            text = 'Advarsel: Din portefølje er braget ned og er intet værd. Nuværende gæld er på ' + str(pp.loc[i-1, 'total_debt']) 
            speech = gTTS(text=text, lang='da', slow=False)
            speech.save("voice.mp3")
            #return_code = subprocess.call(["afplay", 'voice.mp3'])
            #playsound('voice.mp3')
            
            pp.loc[i:, ['savings', 'cash', 'new_equity', 'new_debt', 'nip', 'pv_p', 'interest', 'pv_u', 'tv_u', 'pi_hat', 'g_hat']] = 0
            pp.loc[i:, 'total_debt'] = pp.loc[i-1,'total_debt']
            pp.loc[i:, 'SU_debt'] = pp.loc[i-1, 'SU_debt']
            pp.loc[i:, 'Nordnet_debt'] = pp.loc[i-1, 'Nordnet_debt']
            pp.loc[i:, 'equity'] = pp.loc[i-1, 'equity']
            pp.loc[i:, 'dst'] = pp.loc[i-1, 'dst']
            pp.loc[i:, 'phase'] = pp.loc[i-1, 'phase']
            break
    
    pp['g_hat'] = pp['total_debt']/pp['equity']
    
    return pp

In [ ]:
def calculate100return(savings, market_returns):
     # Running controls
    assert len(savings) == len(market), 'Investment plan should be same no of periods as market' 
    
    columns = ['period', 'savings', 'pv_p', 'market_returns', 'tv_u']
        
    pp = pd.DataFrame(np.empty((len(savings), len(columns))), columns = columns)
    
    pp['period'] = range(len(savings))
    pp['market_returns'] = market_returns
    pp['savings'] = savings
    pp.loc[0, 'market_returns'] = 0
    pp.loc[0, 'pv_p'] = pp.loc[0, 'savings']
    pp.loc[0, 'tv_u'] = pp.loc[0, 'savings']
    
    for i in range(1, len(savings)):
 
        # Period t > 0 primo
        pp.loc[i, 'pv_p'] = pp.loc[i-1, 'tv_u'] + pp.loc[i, 'savings']        
        
        # Period t > 0 ultimo
        pp.loc[i, 'tv_u'] = pp.loc[i, 'pv_p']*(1+pp.loc[i, 'market_returns'])
    
    return pp

In [ ]:
def calculate9050return(savings, market_returns, rf):
    # Strategy where 90% of value is initially invested in stocks, rest in risk free asset
    # Ratio of stocks falls linearly to 50% by age 65 and stays there
    
    # Running controls
    assert len(savings) == len(market), 'Investment plan should be same no of periods as market' 
    
    columns = ['period',  'savings', 'cash', 'pv_p', 'market_returns', 'pv_u', 'tv_u', 'ratio']
        
    pp = pd.DataFrame(np.empty((len(savings), len(columns))), columns = columns)

    pp['period'] = range(len(savings))
    pp['market_returns'] = market_returns
    pp['savings'] = savings
    pp.loc[0, 'market_returns'] = 0
    pp.loc[0, 'pv_p'] = pp.loc[0, 'savings']*0.9
    pp.loc[0, 'cash'] = pp.loc[0, 'savings']*0.1
    pp.loc[0, 'pv_u'] = pp.loc[0, 'pv_p']    
    pp.loc[0, 'tv_u'] = pp.loc[0, 'savings']
    pp.loc[0, 'ratio'] = 90
    
    for i in range(1, len(savings)):
        ratio = max(90 - pp.loc[i, 'period']/12, 50)
        pp.loc[i, 'ratio'] = ratio
        
        # Period t > 0 primo
        pp.loc[i, 'pv_p'] = pp.loc[i-1, 'pv_u'] + pp.loc[i, 'savings']*(ratio/100)        
        pp.loc[i, 'cash'] = pp.loc[i-1, 'cash']*(1+rf) + pp.loc[i, 'savings']*(1-ratio/100)
        
        # Period t > 0 ultimo
        pp.loc[i, 'pv_u'] = pp.loc[i, 'pv_p']*(1+pp.loc[i, 'market_returns'])
        pp.loc[i, 'tv_u'] = pp.loc[i, 'pv_u'] + pp.loc[i, 'cash']
    
    return pp

In [ ]:
savings_year = pd.read_csv('investment_plan_year.csv', sep=';', index_col=0)
savings_year.index = pd.to_datetime(savings_year.index, format='%Y')
savings_month = (savings_year.resample('BMS').pad()/12)['Earnings'].values
savings_year = savings_year['Earnings'].values

In [ ]:
yearly_rf = 0.015
yearly_rm = 0.04  # Weighted average of margin rates

# Set period
period = 'M'


if period == 'M':
    investments = savings_month*0.05
    rf = math.exp(yearly_rf/12)-1
    rm = math.exp(yearly_rm/12)-1
    freq = 'BMS'
else:
    investments = savings_year*0.05
    rf = yearly_rf
    rm = yearly_rm
    freq = 'BYS'


global debt_available
debt_available = {'SU': "", 'Nordnet': ""}
gamma = 2.5
sigma = Market.yearly.pct_change().std()**2
mr = (Market.yearly[-1]/Market.yearly[0])**(1/len(Market.yearly))-1


pi_rf = calc_pi(gamma, sigma, mr, yearly_rf, cost = 0)
pi_rm = calc_pi(gamma, sigma, mr, yearly_rm, cost = 0)

print('pi_rf', pi_rf, 'pi_rm', pi_rm)
    
#market = Market.draw(random_state = 16, freq=period).asfreq(freq, 'pad')
#market = Market.norm_innovations(random_state = 6969, freq=period).asfreq(freq, 'pad')
#market = Market.t_innovations(random_state = 6969, freq=period).asfreq(freq, 'pad')
market = Market.garch(log = False, random_state = 6229, mu_override = 0.030800266141550736).asfreq(freq, 'pad')

      
port = calculate_return(investments, market['Price'].pct_change().values, 10, pi_rf, pi_rm, rf)
port100 = calculate100return(investments, market['Price'].pct_change().values)
port9050 = calculate9050return(investments, market['Price'].pct_change().values, rf)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 10))
vars_to_plot = ['cash', 'total_debt', 'tv_u', '100tv_u', '9050tv_u']
port['100tv_u'] = port100['tv_u']
port['9050tv_u'] = port9050['tv_u']
ax.plot(port.loc[0:200 ,vars_to_plot], linewidth = 2)
ax.plot(port.loc[:200, 'dst'], linestyle="--", color="black", alpha = .5)
#ax.set_yscale('log')
vars_to_plot.append('dst')
ax.legend(vars_to_plot, loc = 'upper left', frameon=False)

In [ ]:
plt.plot(port.loc[:, ['Nordnet_debt', 'SU_debt']])

In [ ]:
plt.plot(port.loc[:, ['g_hat', 'pi_hat']])

In [ ]:
columns = ['period', 'savings', 'cash', 'new_equity', 'new_debt', 'total_debt', 'nip', 'pv_p', 
               'interest', 'market_returns', 'pv_u', 'tv_u', 'dst', 'phase', 'pi_hat', 'ses', 'g_hat']
    
plt.bar(x = port['period'], height = port['phase'])
#plt.xlim([0, 60])

#plt.bar(x=port.loc[0:100, 'period'], height=port.loc[0:100, 'market_returns'])

## Characteristics of strategies

In [ ]:
std = port.loc[:, ['tv_u', '100tv_u', '9050tv_u']].pct_change().std()

horizon = Market.years-1
mean_return = (port.loc[horizon, ['tv_u', '100tv_u', '9050tv_u']]/port.loc[0, ['tv_u', '100tv_u', '9050tv_u']])**(1/horizon)-1

In [ ]:
mean_return

In [ ]:
std

In [ ]:
sharpe = mean_return/std

In [ ]:
sharpe

# Testing many markets

In [ ]:
def sim_market(seed):
    returns = []
    print(seed)
    market = Market.garch(random_state = seed).asfreq(freq, 'pad')
    port = calculate_return(investments, market['Price'].pct_change().values, 1, pi_rf, pi_rm, rf, rm)
    port100 = calculate100return(investments, market['Price'].pct_change().values)
    port9050 = calculate9050return(investments, market['Price'].pct_change().values, rf)
    port['100tv_u'] = port100['tv_u']
    port['9050tv_u'] = port9050['tv_u']
    returns.append(port.loc[Market.years-1, ['tv_u', '100tv_u', '9050tv_u']].values)
    return returns
    

In [ ]:
import concurrent
from multiprocessing import Pool
yearly_rf = 0.02
yearly_rm = 0.03

# Set period
period = 'Y'


if period == 'M':
    investments = savings_month*0.05
    rf = yearly_rf/12
    rm = yearly_rm/12
    freq = 'BMS'
else:
    investments = savings_year*0.05
    rf = yearly_rf
    rm = yearly_rm
    freq = 'BYS'


gamma = 1.5
sigma = 0.20**2
mr = 0.07

pi_rf = calc_pi(gamma, sigma, mr, yearly_rf, cost = 0)
pi_rm = calc_pi(gamma, sigma, mr, yearly_rm, cost = 0)

returns = []



seeds_to_sim = list(range(100))

# multiprocessing
p = Pool()

# saving the restaurant information in a variable
res = p.apply(sim_market, seeds_to_sim)

p.close()
p.join()


In [ ]:
returns = pd.DataFrame(returns, columns = ['tv_u', '100tv_u', '9050tv_u'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 15))
vars_to_plot = ['9050tv_u', '100tv_u', 'tv_u']

bin_num = 20
cum = True
alpha = 0.7
bin_width = 5e6
max_return = 3e8

ax.hist(x = returns['9050tv_u'], bins = int(returns['9050tv_u'].max()/bin_width), cumulative = cum, alpha = alpha)
ax.hist(x = returns['100tv_u'], bins = int(returns['100tv_u'].max()/bin_width), cumulative = cum, alpha = alpha)
ax.hist(x = returns['tv_u'], bins = int(returns['tv_u'].max()/bin_width), cumulative = cum, alpha = alpha)



ax.set_xlim([0, max_return])
ax.legend(vars_to_plot)
#vars_to_plot.append('dst')
#ax.legend(vars_to_plot, loc = 'upper left', frameon=False)